In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#importing necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:


num_mild_tr = '/content/drive/MyDrive/newImg/train/mild'

num_moderate_tr = '/content/drive/MyDrive/newImg/train/moderate'

num_noDR_tr = '/content/drive/MyDrive/newImg/train/noDR'

num_proliferativeDR_tr = '/content/drive/MyDrive/newImg/train/proliferativeDR'

num_severe_tr = '/content/drive/MyDrive/newImg/train/severe'


num_mild_val = '/content/drive/MyDrive/newImg/train/mild'

num_moderate_val = '/content/drive/MyDrive/newImg/train/moderate'

num_noDR_val = '/content/drive/MyDrive/newImg/train/noDR'

num_proliferativeDR_val = '/content/drive/MyDrive/newImg/train/proliferativeDR'

num_severe_val = '/content/drive/MyDrive/newImg/train/severe'



train_directory = num_mild_tr + num_moderate_tr + num_noDR_tr + num_proliferativeDR_tr + num_severe_tr
valid_directory = num_mild_val + num_moderate_val + num_noDR_val + num_proliferativeDR_val + num_severe_val

In [4]:
import os

# Paths for training directories
paths = {
    'mild_tr': '/content/drive/MyDrive/newImg/train/mild',
    'moderate_tr': '/content/drive/MyDrive/newImg/train/moderate',
    'noDR_tr': '/content/drive/MyDrive/newImg/train/noDR',
    'proliferativeDR_tr': '/content/drive/MyDrive/newImg/train/proliferativeDR',
    'severe_tr': '/content/drive/MyDrive/newImg/train/severe',
}

# Function to count images
def count_images(path):
    return len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))])

# Counting and printing the number of images in each directory
for label, path in paths.items():
    print(f"{label} has {count_images(path)} images")


mild_tr has 369 images
moderate_tr has 369 images
noDR_tr has 369 images
proliferativeDR_tr has 295 images
severe_tr has 169 images


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define base directories for training and validation data
train_directory = '/content/drive/MyDrive/newImg/train/'
valid_directory = '/content/drive/MyDrive/newImg/train/'  # Ensure this is the correct path for validation data

# Setup ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for validation data

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(32, 32),  # resize all images to 150x150
    batch_size=32,
    class_mode='categorical'  # use 'categorical' for multi-class classification
)

validation_generator = validation_datagen.flow_from_directory(
    valid_directory,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical'
)


Found 1571 images belonging to 5 classes.
Found 1571 images belonging to 5 classes.


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # 5 classes
])


In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
     epochs=120,            # Number of epochs to train the model
)


Epoch 1/120
50/50 [==============================] - 1256s 25s/step - loss: 1.5833 - accuracy: 0.2400 - val_loss: 1.5705 - val_accuracy: 0.2572
Epoch 2/120
50/50 [==============================] - 516s 10s/step - loss: 1.5406 - accuracy: 0.3049 - val_loss: 1.4813 - val_accuracy: 0.3323
Epoch 3/120
50/50 [==============================] - 512s 10s/step - loss: 1.4681 - accuracy: 0.3686 - val_loss: 1.4589 - val_accuracy: 0.3514
Epoch 4/120
50/50 [==============================] - 509s 10s/step - loss: 1.3931 - accuracy: 0.4042 - val_loss: 1.3571 - val_accuracy: 0.3870
Epoch 5/120
50/50 [==============================] - 521s 11s/step - loss: 1.3798 - accuracy: 0.4208 - val_loss: 1.4333 - val_accuracy: 0.3635
Epoch 6/120
50/50 [==============================] - 521s 11s/step - loss: 1.3265 - accuracy: 0.4398 - val_loss: 1.2575 - val_accuracy: 0.4691
Epoch 7/120
50/50 [==============================] - 504s 10s/step - loss: 1.2943 - accuracy: 0.4424 - val_loss: 1.2689 - val_accuracy: 0.446

In [ ]:
import matplotlib.pyplot as plt
# Assuming 'accuracy' was used as a metric during model compilation
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
plt.plot(epochs, acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

#### Plotting Training and Validation Loss

plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()  # Adjusts plot parameters to give a pleasing layout
plt.show()
